In [3]:
from text_encoder import *
from visual_encoder import *
from drawing import *
from util import *

import os
from easydict import EasyDict
import yaml

import numpy as np
import tensorflow.compat.v1 as tf
from scipy.special import softmax

from matplotlib import pyplot as plt
from matplotlib import patches
import cv2
from PIL import Image

from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
import shutil

print("START UI")

class firstWindow(QWidget):
  def __init__(self, *args, **kwargs):
    super(firstWindow, self).__init__(*args, **kwargs)
    self.image_path = './examples/test.jpg'
    self.setWindowTitle('Detect objects. Please input custom labels.')
    self.currentText = ""
    self.bboxMax = CONFIG.max_boxes_to_draw
    self.category_name_string=""
    
    self.img = cv2.imread(self.image_path)
    self.imageHeight, self.imageWidth, self.bytesPerComponent = self.img.shape
    self.resize(self.imageWidth, self.imageHeight)
    self.move(int((QDesktopWidget().screenGeometry().width()-self.width())/2),
              int((QDesktopWidget().screenGeometry().height()-self.height())/2))  
    
    self.lb_0 = originalLabel(self) 
    self.customLabel = customInputLabel(self)
    self.bboxNumLabel = bboxInputLabel(self)
    self.button_1_Init()
    self.button_2_Init()
    self.button_3_Init()
    self.button_4_Init()
    
  def VisualImage(self):
    #//print(self.image_path)
    self.img = cv2.imread(self.image_path)
    self.imageHeight, self.imageWidth, self.bytesPerComponent = self.img.shape
    self.resize(self.imageWidth, self.imageHeight)
    self.move(int((QDesktopWidget().screenGeometry().width()-self.width())/2),
              int((QDesktopWidget().screenGeometry().height()-self.height())/2)) 
    self.lb_0.setGeometry(QRect(0, 0, self.imageWidth, self.imageHeight))
    bytesPerLine = 3 * self.imageWidth
    cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB, self.img)
    QImg = QImage(self.img.data, self.imageWidth, self.imageHeight, bytesPerLine, QImage.Format_RGB888)
    pixmap = QPixmap.fromImage(QImg)
    self.lb_0.setPixmap(pixmap)
    self.update()
    
  def textchange(self, text):
    self.currentText = text
  def numchange(self, bboxNumbers):
    self.bboxMax = bboxNumbers
    #//print(self.bboxMax)
      
  def showPath(self):
    print(self.image_path)
  
  def button_1_Init(self):
    self.bt = QPushButton(self)
    self.bt.resize(30, 30)
    self.bt.setStyleSheet("background-color:transparent")
    self.bt.setIcon(QIcon("icon/icons8-enter-mac-key-90.png"))
    self.bt.setIconSize(QSize(self.bt.size()))
    self.bt.move(self.customLabel.x()+520, self.lb_0.y()+15)
    self.bt.clicked.connect(self.clickInputCustomLabel)
    
  def clickInputCustomLabel(self):
    self.msg_label = QMessageBox(self)
    self.msg_label.setText("Get custom Labels :  \n" + self.currentText + "\nand Max bbox numbers : \n"+self.bboxMax)
    self.msg_label.setIcon(QMessageBox.Information)
    self.msg_label.show()
    cusLabel = self.currentText.split(',')
    self.category_name_string = ';'.join(cusLabel) ## + background 
    print(self.category_name_string)
    
    with open("./config/demo.yaml") as f:
      doc = yaml.safe_load(f)
    doc['max_boxes_to_draw'] = int(self.bboxMax)
    with open("./config/demo.yaml", 'w') as f:
      yaml.safe_dump(doc, f, default_flow_style=False)
       
  def button_2_Init(self):
    self.bt_2 = QPushButton(self)
    self.bt_2.resize(30, 30)
    self.bt_2.setStyleSheet("background-color:transparent")
    self.bt_2.setIcon(QIcon("icon/play.png"))
    self.bt_2.setIconSize(QSize(self.bt_2.size()))
    self.bt_2.move(self.bt.x()+40, self.bt.y())  
    self.bt_2.clicked.connect(self.startDetecting)
  
  ## WITH VILD MODEL ##
  def startDetecting(self):
    print("START DETECTING!")
    with open("config/demo.yaml", "r") as file:
      CONFIG = yaml.load(file, Loader=yaml.SafeLoader)
      CONFIG = EasyDict(CONFIG)
      
    global max_boxes_to_draw 
    global min_rpn_score_thresh
    global category_indices        
    max_boxes_to_draw = int(self.bboxMax) 
    nms_threshold = CONFIG.nms_threshold
    min_rpn_score_thresh = CONFIG.min_rpn_score_thresh
    min_box_area = CONFIG.min_box_area
    
    category_names = ['background']+[x.strip() for x in self.category_name_string.split(';')]
    categories = [{'name': item, 'id': idx,} for idx, item in enumerate(category_names)]
    category_indices = {cat['id']: cat for cat in categories} 
    
    global model
    text_features = build_text_embedding(categories, model, CONFIG=CONFIG)
    
    global rescaled_detection_boxes 
    global image     
    global valid_indices 
    global detection_roi_scores                   
    (detection_roi_scores,
        detection_boxes,
        detection_masks,
        detection_visual_feat,
        rescaled_detection_boxes,
        valid_indices,
        image_info) = process_image(CONFIG)
      
    image_info = np.squeeze(image_info, axis=0)
    image_height = int(image_info[0, 0])
    image_width = int(image_info[0, 1])

    image = np.asarray(Image.open(open(self.image_path, 'rb')).convert("RGB"))
    assert image_height == image.shape[0]
    assert image_width == image.shape[1]
    
    global isSaved              
    global scores_all      
    isSaved = np.zeros(detection_roi_scores.shape)

    raw_scores = detection_visual_feat.dot(text_features.T)

    if CONFIG.use_softmax:
      scores_all = softmax(CONFIG.temperature * raw_scores, axis=-1)
    else:
      scores_all = raw_scores

    global indices                 
    global indices_fg 
    global segmentations                       
    indices = np.argsort(-np.max(scores_all, axis=1))

    indices_fg = np.array([i for i in indices if np.argmax(scores_all[i]) != 0])  
    
    ymin, xmin, ymax, xmax = np.split(rescaled_detection_boxes, 4, axis=-1)
    processed_boxes = np.concatenate([xmin, ymin, xmax - xmin, ymax - ymin], axis=-1)

    segmentations = paste_instance_masks(detection_masks, processed_boxes, image_height, image_width) 
    
    if len(indices_fg) == 0:
      self.mgb_no = QMessageBox(self)
      self.mgb_no.setIcon(QMessageBox.Information)
      self.mgb_no.setText("ViLD does not detect anything belong to the given category")
      self.mgb_no.show()
      print('ViLD does not detect anything belong to the given category')
    else:
      image_with_detections = visualize_boxes_and_labels_on_image_array( \
        np.array(image),
        rescaled_detection_boxes[indices_fg],
        valid_indices[:max_boxes_to_draw][indices_fg],
        detection_roi_scores[indices_fg],   
        category_indices, #numbered_category_indices,
        user=isSaved[indices_fg], scores_all=scores_all, indices_fg=indices_fg, 
        instance_masks=segmentations[indices_fg],
        use_normalized_coordinates=False,
        max_boxes_to_draw=max_boxes_to_draw, min_score_thresh=min_rpn_score_thresh,
        skip_scores=False, skip_labels=False,
      )
      #//plt.imshow(image_with_detections)  
      
      print("Update Detection Window")
      win_2 = resultWindow(self)
      win_2.show()
    
  def button_3_Init(self):   
    self.bt_3 = QPushButton(self)
    self.bt_3.resize(25, 25)
    self.bt_3.setIcon(QIcon("icon/icons8-image-document-64"))
    self.bt_3.setStyleSheet("background-color:transparent")
    self.bt_3.setIconSize(QSize(self.bt_3.size()))
    self.bt_3.move(self.bt_2.x()+40, self.bt.y())
    self.bt_3.clicked.connect(self.clickChangeImage)
  
  def clickChangeImage(self):
    #//print("Clicked!")
    QFDialog = QFileDialog()
    image_choose, _ = QFDialog.getOpenFileName(self, 
        "Choose a new image to detect.",
        os.getcwd() + "/examples",
        "All Files (*);;Image Files (*.jpg)")
    if image_choose == "":
      print("NO IMAGE CHOSEN!")
      return
    print("New Iamge: ", image_choose, "\n")
    self.image_path = image_choose
    #//image_choose = image_choose.split('/')[-1]
    #//self.image_path = './examples/'+ image_choose
    
    with open("./config/demo.yaml") as f:
        doc = yaml.safe_load(f)
    doc['image_path'] = self.image_path
    with open("./config/demo.yaml", 'w') as f:
        yaml.safe_dump(doc, f, default_flow_style=False)
        
    self.VisualImage()
     
  def button_4_Init(self):   
    self.bt_4 = QPushButton(self)
    self.bt_4.resize(25, 25)
    self.bt_4.setIcon(QIcon("icon/icons8-save-50.png"))
    self.bt_4.setStyleSheet("background-color:transparent")
    self.bt_4.setIconSize(QSize(self.bt_4.size()))
    self.bt_4.move(self.bt_3.x()+40, self.bt.y())
    self.bt_4.clicked.connect(self.saveDetectedRes) 
  
  def saveDetectedRes(self):
    shutil.rmtree("./result")
    os.mkdir('result')
    cnt = 0
    global image                    ########
    global rescaled_detection_boxes ########
    raw_image = np.array(image)
    n_boxes = rescaled_detection_boxes.shape[0]
    global indices                  ########
    global detection_roi_scores     ########
    global scores_all               ########
    global isSaved                  ########
    for anno_idx in indices[0:int(n_boxes)]:
      #//print(rescaled_detection_boxes)
      rpn_score = detection_roi_scores[anno_idx]
      bbox = rescaled_detection_boxes[anno_idx]  ## Get BBox
      scores = scores_all[anno_idx]
      showOrNot = isSaved[anno_idx]
      
      if np.argmax(scores) == 0:
        continue
      if showOrNot==2 or showOrNot==3: # if showOrNot==0 or showOrNot==2:
        continue
      
      y1, x1, y2, x2 = int(np.floor(bbox[0])), int(np.floor(bbox[1])), int(np.ceil(bbox[2])), int(np.ceil(bbox[3]))
      img_w_mask = plot_mask(mask_color, alpha, raw_image, segmentations[anno_idx])
      crop_w_mask = img_w_mask[y1:y2, x1:x2, :]
      
      category_names = ['background']+[x.strip() for x in self.category_name_string.split(';')] 
      fig_size_h = min(max(5, int(len(category_names)/2.5)), 10)
      fig, axs = plt.subplots(
        1, 4, figsize=(fig_size_w, fig_size_h),
        gridspec_kw={'width_ratios': [3, 1, 1, 2]}, 
        constrained_layout=True
      )
      #@ Draw bounding box.
      rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=line_thickness, edgecolor='r', facecolor='none')
      axs[0].add_patch(rect)
      axs[0].set_xticks([])
      axs[0].set_yticks([])
      axs[0].set_title(f'bbox: {y1, x1, y2, x2}   area: {(y2 - y1) * (x2 - x1)}   rpn score: {rpn_score:.4f}')
      axs[0].imshow(raw_image)
      #@ Draw image in a cropped region.
      crop = np.copy(raw_image[y1:y2, x1:x2, :])
      axs[1].set_xticks([])
      axs[1].set_yticks([])
      axs[1].set_title(f'predicted: {category_names[np.argmax(scores)]}')
      showLabel = category_names[np.argmax(scores)]
      axs[1].imshow(crop)
      #@ Draw segmentation inside a cropped region.
      axs[2].set_xticks([])
      axs[2].set_yticks([])
      axs[2].set_title('mask')
      axs[2].imshow(crop_w_mask)
      #@ Draw category scores.
      fontsize = max(min(fig_size_h/float(len(category_names))*45, 20), 8)
      for cat_idx in range(len(category_names)):
        axs[3].barh(cat_idx, scores[cat_idx], 
                    color='orange' if scores[cat_idx] == max(scores) else 'blue')
      axs[3].invert_yaxis()
      axs[3].set_axisbelow(True)
      axs[3].set_xlim(0, 1)
      plt.xlabel("confidence score")
      axs[3].set_yticks(range(len(category_names)))
      axs[3].set_yticklabels(category_names, fontdict={'fontsize': fontsize})

      f = plt.gcf()
      f.savefig('result/result_{id}.jpg'.format(id=anno_idx), bbox_inches="tight")
      f.clear()
      cnt += 1
    self.msg_label = QMessageBox(self)
    self.msg_label.setText("RESULTS SAVED!")
    self.msg_label.setInformativeText("Permitted Detection Counts: " + str(cnt) + '!')
    self.msg_label.setIcon(QMessageBox.Information)
    self.msg_label.show()
  
class originalLabel(QLabel):
  def __init__(self, parent=None):
    super(originalLabel, self).__init__((parent))
    self.parentWindow = parent
    self.setGeometry(QRect(0, 0, self.parentWindow.imageWidth, self.parentWindow.imageHeight))
    
    bytesPerLine = 3 * self.parentWindow.imageWidth
    cv2.cvtColor(self.parentWindow.img, cv2.COLOR_BGR2RGB, self.parentWindow.img)
    QImg = QImage(self.parentWindow.img.data, self.parentWindow.imageWidth, self.parentWindow.imageHeight, bytesPerLine, QImage.Format_RGB888)
    pixmap = QPixmap.fromImage(QImg)
    self.setPixmap(pixmap)
    
class customInputLabel(QLineEdit):
  def __init__(self, parent=None):
    super(customInputLabel, self).__init__((parent))
    self.parentWindow = parent
    
    re = QRegExp('[a-zA-Z, ]+$')
    re_validato = QRegExpValidator(re)
    self.setValidator(re_validato)
    self.move(self.parentWindow.lb_0.x()+50, self.parentWindow.lb_0.y()+20)
    self.resize(510, 20)
    self.setPlaceholderText("Please input your custom label(s) to Zero-shot Detect! [separated by',']")
    #self.setText("Please input your custom label(s) to Zero-shot Detect! [separated by',']")
    self.textChanged.connect(self.parentWindow.textchange) 
    
    def mousePressEvent(self, event):
      if event.buttons () == QtCore.Qt.LeftButton:
        self.setFocus()
    
class bboxInputLabel(QLineEdit):
  def __init__(self, parent=None):
    super(bboxInputLabel, self).__init__((parent))
    self.parentWindow = parent
    re = QRegExp('[0-9]+$')
    re_validato = QRegExpValidator(re)
    self.setValidator(re_validato)
    self.move(self.parentWindow.lb_0.x()+50, self.parentWindow.lb_0.y()+45)
    self.resize(160, 20)
    self.setPlaceholderText("Max bbox numbers: ")
    #self.setText("Max bbox numbers:")
    self.textChanged.connect(self.parentWindow.numchange)   
    
    
class resultWindow(QMainWindow):
  def __init__(self, parent=None):
    super(resultWindow, self).__init__(parent)
    self.parentWindow = parent
    self.setWindowTitle('Detected objects. Zero-shot ends.')
    img = cv2.imread("tmp/ui.jpg")
    self.imageHeight, self.imageWidth, self.bytesPerComponent = img.shape
    self.resize(self.imageWidth, self.imageHeight)
    self.move(int((QDesktopWidget().screenGeometry().width()-self.width())/2),
              int((QDesktopWidget().screenGeometry().height()-self.height())/2))
        
    self.lb = resultLabel(self)
    self.lb.setGeometry(QRect(0, 0, self.imageWidth, self.imageHeight))
    bytesPerLine = 3 * self.imageWidth
    cv2.cvtColor(img, cv2.COLOR_BGR2RGB, img)
    QImg = QImage(img.data, self.imageWidth, self.imageHeight, bytesPerLine, QImage.Format_RGB888)
    pixmap = QPixmap.fromImage(QImg)
    self.lb.setPixmap(pixmap)
    self.lb.setCursor(Qt.CrossCursor)

class resultLabel(QLabel):
  def __init__(self, parent=None):
    super(resultLabel, self).__init__((parent))   
    self.parentWindow = parent
    self.rootWindow = self.parentWindow.parentWindow
    self.x0 = 0
    self.y0 = 0
    self.x1 = 0
    self.y1 = 0
    self.flag = False
    self.index = 0

  def mousePressEvent(self, event):
    self.flag = True
    self.x0 = event.x()
    self.y0 = event.y()
    #//print(self.x0, self.y0)
    self.index = 0
    ## TODO Area Cverlap ## 
    global indices_fg               ########
    global rescaled_detection_boxes ########
    global isSaved                  ########
    for self.index in indices_fg:
      #//print(self.index)
      ymin, xmin, ymax, xmax = rescaled_detection_boxes[self.index]
      #//print(ymin, xmin, ymax, xmax)
      if self.x0 < xmax and self.x0 > xmin and self.y0 < ymax and self.y0 > ymin and isSaved[self.index]!=3:
        self.cb = QComboBox(self)
        self.cb.move(self.x0, self.y0)
        self.cb.addItem('Accept or Reject this result')
        self.cb.addItem(QtGui.QIcon("./icon/icons8-assessments-90.png"),'Accept')
        self.cb.addItem(QtGui.QIcon("./icon/icons8-multiplication-90.png"),'Reject')
        self.cb.addItem(QtGui.QIcon("./icon/icons8-garbage-66.png"),'Discard it FOREVER')
        self.cb.setCurrentIndex(0)
        
        self.cb.currentIndexChanged[str].connect(self.oper)
        self.cb.show()
        break
      
  def oper(self, choice):
    global image                    ########
    global valid_indices            ########
    global max_boxes_to_draw        ########
    global detection_roi_scores     ########
    global category_indices         ########
    global segmentations            ########
    global min_rpn_score_thresh     ########
    #//print(i)
    if choice == "Accept":
      #//print(self.index)
      # Global Variable
      isSaved[self.index] = 1
      self.mgb = QMessageBox(self)
      self.mgb.setIcon(QMessageBox.Information)
      self.mgb.setText("This result has been saved")
      self.mgb.setInformativeText("The result directory would be flushed each time")
      self.mgb.show()
      
    elif choice == "Reject":
      #//print(self.index)
      # Global Variable
      isSaved[self.index] = 2
      self.mgb = QMessageBox(self)
      self.mgb.setIcon(QMessageBox.Warning)
      self.mgb.setText("This result has been discarded")
      self.mgb.setInformativeText("Later, it will not be saved")
      self.mgb.show()
    
    elif choice == "Discard it FOREVER":
      isSaved[self.index] = 3
      self.mgb = QMessageBox(self)
      self.mgb.setIcon(QMessageBox.Warning)
      self.mgb.setText("Discard FOREVER")
      self.mgb.setInformativeText("CANNOT RECOVER!")
      self.mgb.show()
      
    self.cb.deleteLater()
    # Update Results
    image_with_detections = visualize_boxes_and_labels_on_image_array( \
      np.array(image),
      rescaled_detection_boxes[indices_fg],
      valid_indices[:max_boxes_to_draw][indices_fg],
      detection_roi_scores[indices_fg],   
      category_indices, #numbered_category_indices,
      user=isSaved[indices_fg], scores_all=scores_all, indices_fg=indices_fg, 
      instance_masks=segmentations[indices_fg],
      use_normalized_coordinates=False,
      max_boxes_to_draw=max_boxes_to_draw, min_score_thresh=min_rpn_score_thresh,
      skip_scores=False, skip_labels=False,
    )
    
    img = cv2.imread("tmp/ui.jpg")
    height, width, bytesPerComponent = img.shape
    self.setGeometry(QRect(0, 0, width, height))
    bytesPerLine = 3 * width
    cv2.cvtColor(img, cv2.COLOR_BGR2RGB, img)
    QImg = QImage(img.data, width, height, bytesPerLine, QImage.Format_RGB888)
    pixmap = QPixmap.fromImage(QImg)
    self.setPixmap(pixmap)
    self.setCursor(Qt.CrossCursor)
    
  def mouseReleaseEvent(self,event):
    self.flag = False




import pdb
####### Main Function ######
def main(CONFIG):
  clip.available_models()
  global model
  model, preprocess = clip.load("ViT-B/32")
  if not CONFIG.run_on_gpu:
    model.cpu()
  
  app = QApplication(sys.argv)
  win_1 = firstWindow()
  
  win_1.show()
  app.exec_()
  
  
  
  
# Running  
with open("config/demo.yaml", "r") as file:
  CONFIG = yaml.load(file, Loader=yaml.SafeLoader)
  CONFIG = EasyDict(CONFIG)
main(CONFIG)

START UI
